In [1]:
def ewd(x, y):  
    """Apply Equal Width Discretization (EWD) to the training data to determine variances"""
    
    df_train = np.stack((x, y)).T
    np.matrix(df_train)
    df_train = df_train[np.argsort(df_train[:,0])]
    cuts1, cuts2 = pd.cut(df_train[:,0], nbins, retbins=True)
    
    return df_train, cuts1, cuts2

def binned_statistics(x,y):
    """Find the mean, variance and number of counts within the bins described by ewd"""
    
    df_train, cuts1, cuts2 = ewd(x,y)
    mean, edges, binnum = scipy.stats.binned_statistic(df_train[:,0], df_train[:,1], statistic='mean', bins=cuts2)
    std, edges, binnum = scipy.stats.binned_statistic(df_train[:,0], df_train[:,1], statistic='std', bins=cuts2)
    count, edges, binnum = scipy.stats.binned_statistic(df_train[:,0], df_train[:,1], statistic='count', bins=cuts2)
    
    return mean, std, count

def vectorize_variance(x,y):
    """Apply the binned variances to the original training data"""
    
    df_train, cuts1, cuts2 = ewd(x,y)
    mean, std, count = binned_statistics(x,y)
    variance=[]
    m=0
    i=0
    while i<len(count):
        maximum = count[i]

        while m < maximum:
            variance.append(std[i])
            m+=1
        else:
            m=0
            i+=1
    return np.array(variance)

def vectorize_mean(x,y):
    
    df_train, cuts1, cuts2 = ewd(x,y)
    mean, std, count = binned_statistics(x,y)
    means=[]
    m=0
    i=0
    while i<len(count):
        maximum = count[i]

        while m < maximum:
            means.append(mean[i])
            m+=1
        else:
            m=0
            i+=1
    return np.array(means)

def plot_uncertainties(x, y, nbins):
    nbins
    df_train, cuts1, cuts2 = ewd(x, y)
    length = len(df_train[:,0])
    mean, std, count = binned_statistics(x,y)
    variance_vector = vectorize_variance(x,y)
    mean_vector = vectorize_mean(x,y)
    
    upper_limit = mean_vector + variance_vector
    lower_limit = mean_vector - variance_vector

    ## Plot the output
    plt.plot(np.linspace(-0.12, 0.123, nbins), mean, label='Mean')
    plt.fill_between(df_train[:,0], upper_limit, lower_limit, color='lightblue', label='Uncertainty (std)')
    plt.legend()
    
"""Neural network functions: """
    
def custom_cost(y_true, y_pred):
    '''Chi square function'''
    return tf.reduce_mean(tf.square((y_true-y_pred)/sigma))

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def bootstrap():
    df_train_a, df_train_b = train_test_split(df_train, test_size=0.5)
    df_train_1, df_train_2 = train_test_split(df_train_a, test_size=0.5)
    df_train_3, df_train_4 = train_test_split(df_train_b, test_size=0.5)
    
    return df_train_1, df_train_2, df_train_3, df_train_4
    
    
def smooth(x,window_len=10,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """
    
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]

    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    
    index = int(window_len/2)
    return y[(index-1):-(index)]

    

